In [11]:
import cv2
import open3d as o3d
import depthai as dai
from slam import process
from display import Display
from pointmap import PointMap


In [3]:

# Create a pipeline
pipeline = dai.Pipeline()

# Define a source - color camera
camRgb = pipeline.createColorCamera()
camRgb.setBoardSocket(dai.CameraBoardSocket.RGB)
camRgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_4_K)
camRgb.setVideoSize(1000, 1000)
camRgb.setInterleaved(False)
camRgb.setColorOrder(dai.ColorCameraProperties.ColorOrder.RGB)

# Create outputs
xoutRgb = pipeline.createXLinkOut()
xoutRgb.setStreamName("rgb")
camRgb.preview.link(xoutRgb.input)

xoutRgbVideo = pipeline.createXLinkOut()
xoutRgbVideo.setStreamName("video")
xoutRgbVideo.input.setBlocking(False)
xoutRgbVideo.input.setQueueSize(1)
camRgb.video.link(xoutRgbVideo.input)

# Create objects for point mapping and display
pmap = PointMap()
display = Display()
pcd = o3d.geometry.PointCloud()
visualizer = o3d.visualization.Visualizer()
visualizer.create_window(window_name="3D plot", width=960, height=540)

# Connect to the device
with dai.Device() as device:
    print('Connected cameras: ', device.getConnectedCameras())
    device.startPipeline(pipeline)

    # Output queues
    qRgb = device.getOutputQueue(name="rgb", maxSize=30, blocking=False)
    out = device.getOutputQueue(name="video", maxSize=1, blocking=False)

    while True:
        frame = qRgb.get()
        output = out.get()
        frame = output.getCvFrame()

        img, tripoints, kpts, matches = process(frame)
        xyz = pmap.collect_points(tripoints)

        if kpts is not None or matches is not None:
            display.display_points2d(frame, kpts, matches)

        display.display_vid(frame)

        if xyz is not None:
            display.display_points3d(xyz, pcd, visualizer)

        key = cv2.waitKey(1)
        if key == ord('q'):
            break

cv2.destroyAllWindows()


C:\Users\azuma\AppData\Local\Temp\ipykernel_35508\2085936821.py:13: DeprecationWarning: RGB is deprecated, use CAM_A or address camera by name instead.
  camRgb.setBoardSocket(dai.CameraBoardSocket.RGB)


Connected cameras:  [<CameraBoardSocket.CAM_A: 0>, <CameraBoardSocket.CAM_B: 1>, <CameraBoardSocket.CAM_C: 2>]


: 